
TRABAJO PRÁCTICO PYTHON - ANÁLISIS DE VENTAS Y MARKETING

Alumno: Erica Ramos

Entorno: Google Colab

Objetivo: Analizar cómo las ventas de diferentes productos se relacionan con las características del cliente y las campañas de marketing.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === ETAPA 1: RECOPILACIÓN Y PREPARACIÓN DE DATOS ===

ventas = pd.read_csv('ventas.csv')
clientes = pd.read_csv('clientes.csv')
marketing = pd.read_csv('marketing.csv')

print("=== DATASET: VENTAS ===")
print(ventas.head())
print("\n=== DATASET: CLIENTES ===")
print(clientes.head())
print("\n=== DATASET: MARKETING ===")
print(marketing.head())

ventas['fecha_venta'] = pd.to_datetime(ventas['fecha_venta'], dayfirst=True, errors='coerce')
ventas['precio'] = ventas['precio'].replace('[\\$,]', '', regex=True).astype(float)
ventas['monto_venta'] = ventas['precio'] * ventas['cantidad']

ventas_mensuales = ventas.groupby(ventas['fecha_venta'].dt.to_period('M'))['monto_venta'].sum().reset_index()
ventas_mensuales.rename(columns={'fecha_venta': 'mes', 'monto_venta': 'total_ventas'}, inplace=True)

print("\n=== VENTAS MENSUALES ===")
print(ventas_mensuales)

ventas_lista = ventas[['producto', 'precio', 'cantidad']].to_dict('records')
print("\nEjemplo de estructura de datos (lista de diccionarios):")
print(ventas_lista[:5])

print("\n=== INFORMACIÓN DE LOS DATASETS ===")
for nombre, df in zip(['ventas', 'clientes', 'marketing'], [ventas, clientes, marketing]):
    print(f"\n--- {nombre.upper()} ---")
    print(df.info())
    print(df.describe(include='all'))

print("\n=== CALIDAD DE DATOS ===")
for nombre, df in zip(['ventas', 'clientes', 'marketing'], [ventas, clientes, marketing]):
    print(f"\n--- {nombre.upper()} ---")
    print("Valores nulos por columna:")
    print(df.isnull().sum())
    print("Duplicados:", df.duplicated().sum())

# === ETAPA 2: PREPROCESAMIENTO Y LIMPIEZA ===

ventas_clean = ventas.drop_duplicates()
clientes_clean = clientes.drop_duplicates()
marketing_clean = marketing.drop_duplicates()

for df in [ventas_clean, clientes_clean, marketing_clean]:
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.replace(r'[^\w\s\.,-]', '', regex=True)

print("\nDuplicados eliminados y texto limpiado.")

umbral = ventas_clean['monto_venta'].quantile(0.8)
productos_top = ventas_clean[ventas_clean['monto_venta'] >= umbral]
print("\n=== PRODUCTOS DE ALTO RENDIMIENTO ===")
print(productos_top[['producto', 'monto_venta']].head())

if 'categoria' in ventas_clean.columns:
    resumen_categoria = ventas_clean.groupby('categoria')['monto_venta'].sum().reset_index().sort_values(by='monto_venta', ascending=False)
    print("\n=== RESUMEN POR CATEGORÍA ===")
    print(resumen_categoria)

if 'producto' in ventas_clean.columns and 'producto' in marketing_clean.columns:
    ventas_marketing = pd.merge(ventas_clean, marketing_clean, on='producto', how='left')
    print("\n=== DATOS INTEGRADOS (VENTAS + MARKETING) ===")
    print(ventas_marketing.head())
else:
    print("\nNo se pudo integrar: no hay columna común entre ventas y marketing.")

# === ETAPA 3: VISUALIZACIÓN DE DATOS ===
print("\n=== VISUALIZACIÓN DE DATOS ===")

# --- Limpieza previa ---
ventas['monto_venta'] = pd.to_numeric(ventas['monto_venta'], errors='coerce')
ventas_mensuales['total_ventas'] = (
    ventas_mensuales['total_ventas']
    .astype(str)
    .replace('[^0-9\\.-]', '', regex=True)
    .astype(float)
)
ventas_mensuales['mes'] = ventas_mensuales['mes'].astype(str)
ventas_mensuales = ventas_mensuales.sort_values('mes')

print("✅ Columnas numéricas listas para graficar.")

# 1. Ventas mensuales
plt.figure(figsize=(8,4))
sns.lineplot(data=ventas_mensuales, x='mes', y='total_ventas', marker='o')
plt.title('Evolución de Ventas Mensuales')
plt.xlabel('Mes')
plt.ylabel('Total de Ventas ($)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# 2. Ventas por categoría
if 'categoria' in ventas_clean.columns:
    plt.figure(figsize=(7,4))
    sns.barplot(data=resumen_categoria, x='categoria', y='monto_venta', palette='viridis')
    plt.title('Ventas Totales por Categoría')
    plt.xlabel('Categoría')
    plt.ylabel('Monto Total ($)')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("⚠️ No se encontró la columna 'categoria' para graficar ventas por categoría.")

# 3. Top productos
plt.figure(figsize=(8,4))
top_prod = productos_top.groupby('producto')['monto_venta'].sum().sort_values(ascending=False).head(10)
sns.barplot(x=top_prod.values, y=top_prod.index, palette='mako')
plt.title('Top 10 Productos por Monto de Venta')
plt.xlabel('Monto Total ($)')
plt.ylabel('Producto')
plt.show()

# 4. Relación ingresos del cliente y monto de venta
if 'id_cliente' in ventas_clean.columns and 'id_cliente' in clientes_clean.columns:
    ventas_clientes = pd.merge(ventas_clean, clientes_clean, left_on='id_cliente', right_on='id_cliente', how='left')
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=ventas_clientes, x='ingresos', y='monto_venta', alpha=0.6)
    plt.title('Relación entre Ingresos del Cliente y Monto de Venta')
    plt.xlabel('Ingresos del Cliente ($)')
    plt.ylabel('Monto de Venta ($)')
    plt.show()
else:
    print("⚠️ No se pudo graficar la relación ingresos–ventas: falta 'id_cliente' en alguno de los datasets.")

# === CONCLUSIONES ===
print("\n✅ Conclusiones sugeridas:")
print("1. Los meses con mayor volumen de ventas podrían coincidir con campañas de marketing activas.")
print("2. Los productos de alto rendimiento representan el 20% superior en ventas totales.")
print("3. Las categorías más rentables deben priorizarse en futuras campañas.")
print("4. Los clientes con mayores ingresos tienden a realizar compras de mayor monto.")




# 🧾 ANEXO

### 🔹 Archivos utilizados:
- **ventas.csv** → Contiene los registros de ventas individuales, con las columnas:
  `id_venta`, `producto`, `precio`, `cantidad`, `fecha_venta`, `categoria`.
  Se usó para calcular el monto total de ventas y realizar los análisis de rendimiento por producto y categoría.

- **clientes.csv** → Incluye información demográfica de los clientes, con columnas:
  `id_cliente`, `nombre`, `edad`, `ciudad`, `ingresos`.
  Se utilizó para analizar el perfil de clientes y, potencialmente, relacionar ingresos con montos de venta.

- **marketing.csv** → Registra las campañas publicitarias, con columnas:
  `id_campanha`, `producto`, `canal`, `costo`, `fecha_inicio`, `fecha_fin`.
  Sirvió para integrar información de marketing y ventas y analizar posibles impactos de las campañas en los resultados.

---

### 📊 Descripción del proceso:
1. **Recopilación de datos:** Se cargaron los tres archivos y se verificó su estructura.
2. **Limpieza y preprocesamiento:**  
   - Eliminación de duplicados y caracteres especiales.  
   - Conversión de columnas a tipos adecuados (por ejemplo, fechas y precios).  
   - Cálculo de nuevas variables, como `monto_venta`.  
3. **Análisis exploratorio y visualización:**  
   - Ventas mensuales y por categoría.  
   - Productos con mejor rendimiento.  
   - Integración con campañas de marketing.  
4. **Conclusiones:**  
   - Se identificaron los productos más rentables y los meses de mayor actividad.  
   - Las campañas de marketing se pueden alinear con las categorías de mayor volumen de venta.

---

### 🧠 Observaciones finales:
El análisis demuestra cómo las variables de producto, cliente y marketing se pueden combinar para entender las dinámicas de ventas y optimizar futuras estrategias comerciales.
